In [ ]:
from keras.models import load_model
from keras.models import Model
from keras.layers import Input, Lambda
from keras.layers import Conv2DTranspose, BatchNormalization, Activation, Dropout
from keras.optimizers import Adam
import keras.backend as K
import tensorflow as tf
import numpy as np
from numpy import asarray
from numpy import expand_dims
from numpy import linalg
from PIL import Image
from imageio import imread
from skimage.transform import resize
from scipy.spatial import distance
import os
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def prewhiten(x):

    if K.ndim(x) == 4:
        axis = (1, 2, 3)
        size = K.int_shape(x)
        size = size[1]*size[2]*size[3]
    elif K.ndim(x) == 3:
        axis = (0, 1, 2)
        size = K.int_shape(x)
        size = size[0]*size[1]*size[2]
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = K.mean(x, axis=axis, keepdims=True)
    std = K.std(x, axis=axis, keepdims=True)
    std_adj = K.maximum(std, 1.0/np.sqrt(size)) 
    y = (x - mean) / std_adj
    return y

In [ ]:
# Load Dev data
X_eval = np.load('X_test.npy')
Y_eval = np.load('Y_test.npy')

In [ ]:
X_eval = X_eval[10:20,:,:,:]
Y_eval = Y_eval[10:20,:]

In [ ]:
# Check the data sizes
print(X_eval.shape)
print(Y_eval.shape)

In [ ]:
# Load the FaceNet Model
model_path = './model/keras/model/facenet_keras.h5'
model_FN = load_model(model_path)

In [ ]:
model_path = './model_final_sigmoid_N8_080.h5'
model_final = load_model(model_path, custom_objects={"prewhiten":prewhiten})

In [ ]:
#1. Generate Yhat of Dev set
Yhat_eval = model_final.predict(X_eval)

In [ ]:
#2 Generate YHattilt of navie upsampling
length = X_eval.shape[0]

X_eval_LRHR_np = np.ndarray(shape=(length, 160,160,3), dtype=int)

for i in range(length):
    X_eval_LR = Image.fromarray(X_eval[i])
    X_eval_LRHR = X_eval_LR.resize((160,160), resample=Image.BICUBIC)
    X_eval_LRHR_np[i] = asarray(X_eval_LRHR)

In [ ]:
# 3. Pass through the sub-sequent network
kvar = K.variable(value=X_eval_LRHR_np)
temp1 = Lambda(lambda x: prewhiten(x))(kvar)
temp2 = model_FN(temp1)
temp3 = Lambda(lambda  x: K.l2_normalize(x,axis=1))(temp2)
Yhattilt_eval = K.eval(temp3)

In [ ]:
print(Y_eval.shape)
print(Yhat_eval.shape)
print(Yhattilt_eval.shape)

In [ ]:
Error_Yhat = np.linalg.norm(Y_eval-Yhat_eval, axis=1, keepdims=True)

In [ ]:
Error_Yhattilt = np.linalg.norm(Y_eval-Yhattilt_eval, axis=1, keepdims=True)

In [ ]:
A = Error_Yhat > Error_Yhattilt

In [ ]:
print("Number of sample model performs worse: ", np.sum(A))
print("Number of sample model performs better: ", Error_Yhat.shape[0]-np.sum(A))